In [1]:
#TO DO: need to adjust for events-run matching 
#TO DO: need to change the onsets of various events based on subject response and changes in the trial depending on successful/unsuccesful stops
#should read SST paper about ABCD study to better understand this...



In [ ]:
import os
import pandas as pd
import glob
import re

In [2]:
os.getcwd()

'/net/vast-storage.ib.cluster/scratch/scratch/Fri/dclb/mmc/code/preprocessing/events_file_population'

In [5]:
#note that I am first doing this just for one subject, one run, and one session, and will then loop later once this works! 

In [6]:
# #don't need this anymore! Leaving here for reference for now :) 
# MJ_Go = pd.read_csv ("../../sourcedata/behavioral/HC_009/baseline/SST_R1/MJ_Go.txt", sep = '\t',header = None)
# MJ_SuccStop = pd.read_csv ("../../sourcedata/behavioral/HC_009/baseline/SST_R1/MJ_SuccStop.txt", sep = '\t',header = None)
# MJ_UnsuccStop = pd.read_csv ("../../sourcedata/behavioral/HC_009/baseline/SST_R1/MJ_UnsuccStop.txt", sep = '\t',header = None)
# N_Go = pd.read_csv ("../../sourcedata/behavioral/HC_009/baseline/SST_R1/N_Go.txt", sep = '\t',header = None)
# N_SuccStop = pd.read_csv ("../../sourcedata/behavioral/HC_009/baseline/SST_R1/N_SuccStop.txt", sep = '\t',header = None)
# N_UnsuccStop = pd.read_csv ("../../sourcedata/behavioral/HC_009/baseline/SST_R1/N_UnsuccStop.txt", sep = '\t',header = None)
# grp, num = re.split('(\d+)', 'sub-HC006'.split("-")[-1])[0:2]

In [40]:
# subj_grp="MM"
# subj_num="129"
# session="baseline"
# run_num="1"
# SST_stim_locations_indiv = (glob.glob("../../sourcedata/behavioral/{}_{}/{}/SST_R{}/*.txt".format(subj_grp,subj_num,session,run_num))) 
# print(SST_stim_locations_indiv)



In [41]:
# dfs = []
# for loc in SST_stim_locations_indiv:
#     trial_type = loc.split("/")[-1].split(".")[0]
#     df = pd.read_csv (loc, sep='\t+',header = None,engine='python')
#     numrows_df = df.shape[0]
#     df[2]=[trial_type for val in range(0,numrows_df)]
#     dfs.append(df)
# all_events = pd.concat(dfs, ignore_index=True)
# all_events.set_axis(['onset', 'duration', 'trial_type'], axis=1,inplace=True)


# all_events = pd.concat([all_events]*2)
# all_events.sort_values(by='onset', inplace=True, ascending=True)
# all_events.reset_index(drop=True, inplace=True)

# all_events.loc[0::2].onset +=0.2
# all_events.loc[0::2].trial_type=all_events.loc[0::2].trial_type + '_image'

# all_events.loc[1::2].onset +=1
# all_events.loc[1::2].duration +=0.5
# all_events.loc[1::2].trial_type=all_events.loc[1::2].trial_type + '_arrow'


# display(all_events)

In [7]:
# #writing to all applicable events.tsv files (moco and unco) for a particular grp, num, ses, run 

# events_files_indiv_locations = (glob.glob("../../sub-{}{}/ses-{}/func/*sst*run-0{}*events.tsv".format(subj_grp,subj_num,session,run)))
# for file in events_files_indiv_locations:
#     all_events.to_csv(file, index = False, header=True, sep='\t')

In [8]:
# subj_nums_locations = (glob.glob("../../sub-*"))
# subj_nums = list((re.split('(\d+)', num.split("/")[-1].split("-")[-1])[0:2] for num in subj_nums_locations))
# sessions = ['baseline','1year']
# runs = ['1','2','3']

In [9]:
# for grp, num, ses, run in subj_nums:
    

In [10]:
# ses_locations = (glob.glob("../../sub-{}{}/ses-*".format('HC','009')))
# print(ses_locations)

In [3]:
def make_events_file(SST_stim_locations): 
    dfs = []
    for loc in SST_stim_locations:
        trial_type = loc.split("/")[-1].split(".")[0]
        df = pd.read_csv (loc, sep='\t+',header = None,engine='python')
        numrows_df = df.shape[0]
        df[2]=[trial_type for val in range(0,numrows_df)]
        dfs.append(df)
    all_events = pd.concat(dfs, ignore_index=True)
    all_events.set_axis(['onset', 'duration', 'trial_type'], axis=1,inplace=True)
    all_events.sort_values(by='onset', inplace=True, ascending=True)
    
    #this duplicates every row and puts duplicates one after the other
    all_events = pd.concat([all_events]*2)
    all_events.sort_values(by='onset', inplace=True, ascending=True)
    all_events.reset_index(drop=True, inplace=True)

    #this adds 0.2 to the onset of the first occurence of every row + _image to the title of these rows
    all_events.loc[0::2].onset +=0.2
    all_events.loc[0::2].trial_type+= '_image'
    
    #this adds 1.0 to the onset of the second occurence of every row + 0.5 to the duration + _arrow to the title of these rows
    all_events.loc[1::2].onset +=1
    all_events.loc[1::2].duration +=0.5
    all_events.loc[1::2].trial_type+= '_arrow'
    
    return all_events

In [4]:
events_file_locations = (glob.glob("../../../sub-*/ses-*/func/*sst*run-*events.tsv"))
events_dict={}
for events_loc in events_file_locations:
    split_loc = events_loc.split('/')
    grp, num = re.split('(\d+)', split_loc[3].split("-")[-1])[0:2]
    ses = split_loc[4].split('-')[-1]
    run = split_loc[-1].split('_')[-2].split('-')[-1]
    if run != '3':
        if (grp+num+ses+run not in events_dict.keys()):
            SST_stim_locations = (glob.glob("../../../sourcedata/behavioral/{}_{}/{}/SST_R{}/*.txt".format(grp,num,ses,run)))
            all_events = make_events_file(SST_stim_locations)
            events_dict[grp+num+ses+run]=all_events
        else:
            all_events = events_dict[grp+num+ses+run]
        all_events.to_csv(events_loc, index = False, header=True, sep='\t', float_format='%.3f')
        

../../../sourcedata/behavioral/HC_006/baseline/SST_R1/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_006/baseline/SST_R2/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_007/baseline/SST_R1/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_007/baseline/SST_R2/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_008/baseline/SST_R1/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_008/baseline/SST_R2/MJ_SuccStop.txt


/tmp/ipykernel_61047/1017212140.py:10: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  all_events.set_axis(['onset', 'duration', 'trial_type'], axis=1,inplace=True)
/tmp/ipykernel_61047/1017212140.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_events.loc[0::2].onset +=0.2
/tmp/ipykernel_61047/1017212140.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_events.loc[0::2].trial_type+=

../../../sourcedata/behavioral/HC_009/baseline/SST_R1/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_009/baseline/SST_R2/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_010/baseline/SST_R1/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_010/baseline/SST_R2/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_011/baseline/SST_R1/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_011/baseline/SST_R2/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_012/baseline/SST_R1/MJ_SuccStop.txt


/tmp/ipykernel_61047/1017212140.py:10: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  all_events.set_axis(['onset', 'duration', 'trial_type'], axis=1,inplace=True)
/tmp/ipykernel_61047/1017212140.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_events.loc[0::2].onset +=0.2
/tmp/ipykernel_61047/1017212140.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_events.loc[0::2].trial_type+=

../../../sourcedata/behavioral/HC_012/baseline/SST_R2/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_013/baseline/SST_R1/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_013/baseline/SST_R2/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_014/baseline/SST_R1/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_014/baseline/SST_R2/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_016/baseline/SST_R1/MJ_SuccStop.txt
../../../sourcedata/behavioral/HC_016/baseline/SST_R2/MJ_SuccStop.txt


...

/tmp/ipykernel_61047/1017212140.py:10: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  all_events.set_axis(['onset', 'duration', 'trial_type'], axis=1,inplace=True)
/tmp/ipykernel_61047/1017212140.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_events.loc[0::2].onset +=0.2
/tmp/ipykernel_61047/1017212140.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_events.loc[0::2].trial_type+=

../../../sourcedata/behavioral/MM_315/baseline/SST_R2/MJ_SuccStop.txt
../../../sourcedata/behavioral/MM_330/1year/SST_R1/MJ_SuccStop.txt
../../../sourcedata/behavioral/MM_330/1year/SST_R2/MJ_SuccStop.txt
../../../sourcedata/behavioral/MM_330/baseline/SST_R1/MJ_SuccStop.txt
../../../sourcedata/behavioral/MM_330/baseline/SST_R2/MJ_SuccStop.txt
../../../sourcedata/behavioral/MM_334/1year/SST_R1/MJ_SuccStop.txt
../../../sourcedata/behavioral/MM_334/1year/SST_R2/MJ_SuccStop.txt


/tmp/ipykernel_61047/1017212140.py:10: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  all_events.set_axis(['onset', 'duration', 'trial_type'], axis=1,inplace=True)
/tmp/ipykernel_61047/1017212140.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_events.loc[0::2].onset +=0.2
/tmp/ipykernel_61047/1017212140.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_events.loc[0::2].trial_type+=

../../../sourcedata/behavioral/MM_334/baseline/SST_R1/MJ_SuccStop.txt
../../../sourcedata/behavioral/MM_334/baseline/SST_R2/MJ_SuccStop.txt
../../../sourcedata/behavioral/MM_340/baseline/SST_R1/MJ_SuccStop.txt
../../../sourcedata/behavioral/MM_340/baseline/SST_R2/MJ_SuccStop.txt


/tmp/ipykernel_61047/1017212140.py:10: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  all_events.set_axis(['onset', 'duration', 'trial_type'], axis=1,inplace=True)
/tmp/ipykernel_61047/1017212140.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_events.loc[0::2].onset +=0.2
/tmp/ipykernel_61047/1017212140.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_events.loc[0::2].trial_type+=

In [11]:
events_file_locations = (glob.glob("../../../sub-*/ses-*/func/*sst*run-*events.tsv"))
events_file_locations[0]

'../../../sub-HC006/ses-baseline/func/sub-HC006_ses-baseline_task-sst_rec-moco_run-1_events.tsv'

In [12]:
events_loc = events_file_locations[0]
split_loc = events_loc.split('/')
grp, num = re.split('(\d+)', split_loc[3].split("-")[-1])[0:2]
print(num)

006


In [17]:
ses = split_loc[4].split('-')[-1]
run = split_loc[-1].split('_')[-2].split('-')[-1]
run

'1'